<a href="https://colab.research.google.com/github/juliaradula/teste2/blob/main/Trabalhando_com_dados_de_transcriptoma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transcriptoma

Esse notebook pode ser usado para juntar os dados de anotação funcional  (Trinotate) com classe enzimática (UniprotKB) e abundância num arquivo único. No final há código que pode ser utilizado para criar um documento novo com a classe de enzima desejada.<br>

A esquerda, clique no ícone de pasta e permita acesso ao seu google drive onde devem estar os arquivos que serão utilizados.<br>

Antes de iniciar, clique em ```File``` no canto superior esquerdo e crie uma cópia desse notebook. Utilize a cópia para executar os códigos. Execute uma célula de cada vez, em ordem, clicando na célula e apertando ```shift + enter```.<br>

--<br>
Julia Raspante Martins<br>
Atualizado em 26<wbr>/1<wbr>/2021

In [ ]:
#importar bibliotecas que serao utiizadas
import pandas as pd
import numpy as np
from google.colab import files

In [ ]:
#carregar dados de anotacao funcional (Trinotate)
#os arquivos estao no google drive
#o endereco do arquivo pode ser copiado clicando nos tres pontinhos
#a direita do nome do arquivo e selecionar 'copy path'
#aqui o arquivo selecionado eh 'Trinotate.RMA.xls'
#para usar outro arquivo substitua '/content/drive/MyDrive/RMA/Trinotate.RMA.xls'
#pelo endereco desejado

trinotate = pd.read_csv("/content/drive/MyDrive/RMA/Trinotate.RMA.xls", sep="\t")

#o codigo seguinte '.head()' retorna as primeiras 5 linhas do arquivo
trinotate.head()

,#gene_id,transcript_id,sprot_Top_BLASTX_hit,RNAMMER,prot_id,prot_coords,sprot_Top_BLASTP_hit,Pfam,SignalP,TmHMM,eggnog,Kegg,gene_ontology_blast,gene_ontology_pfam,transcript,peptide
0,TRINITY_DN5078_c0_g1,TRINITY_DN5078_c0_g1_i1,.,.,TRINITY_DN5078_c0_g1_i1.p1,2-628[+],.,.,.,.,.,.,.,.,.,.
1,TRINITY_DN5078_c0_g1,TRINITY_DN5078_c0_g1_i2,.,.,TRINITY_DN5078_c0_g1_i2.p1,2-673[+],.,.,.,.,.,.,.,.,.,.
2,TRINITY_DN5046_c0_g1,TRINITY_DN5046_c0_g1_i1,"PAPOB_HUMAN^PAPOB_HUMAN^Q:263-15,H:61-146^38.3...",.,.,.,.,.,.,.,COG5186^polyA polymerase,KEGG:hsa:56903`KO:K14376,GO:0005737^cellular_component^cytoplasm`GO:000...,.,.,.
3,TRINITY_DN5074_c0_g1,TRINITY_DN5074_c0_g1_i1,.,.,TRINITY_DN5074_c0_g1_i1.p1,6-335[-],.,.,.,.,.,.,.,.,.,.
4,TRINITY_DN5003_c0_g1,TRINITY_DN5003_c0_g1_i1,.,.,TRINITY_DN5003_c0_g1_i1.p1,71-604[+],.,.,.,.,.,.,.,.,.,.


In [ ]:
#para conferir o tamanho da tabela
#nesse caso tem 69080 linhas e 16 colunas
trinotate.shape

(69080, 16)

In [ ]:
#carregar dados de abundancia
#assim como anteriormente, substitua o endereco entre aspas pelo
#endereco do arquivo desejado
abundancia = pd.read_csv("/content/drive/MyDrive/RMA/RMA.abundance.tsv", sep="\t")
abundancia.head()

,target_id,length,eff_length,est_counts,tpm
0,TRINITY_DN5078_c0_g1_i1,629,479.808,11.47500,1.275230
1,TRINITY_DN5078_c0_g1_i2,675,525.785,15.52500,1.574440
2,TRINITY_DN5046_c0_g1_i1,595,445.808,20.00000,2.392140
3,TRINITY_DN5074_c0_g1_i1,583,433.808,18.00000,2.212480
4,TRINITY_DN5003_c0_g1_i1,685,535.785,4.16427,0.414431


In [ ]:
#conferindo o tamanho da tabela
#importante ter o mesmo numero de linhas que a tabela 'trinotate'
abundancia.shape

(69080, 5)

In [ ]:
#trocar o nome das colunas da tabela 'abundancia'
#isso facilitara a mesclagem das tabelas
abundancia.columns = ["transcript_id", "length", "eff_length", "est_counts", "tpm"]
abundancia.head()

,transcript_id,length,eff_length,est_counts,tpm
0,TRINITY_DN5078_c0_g1_i1,629,479.808,11.47500,1.275230
1,TRINITY_DN5078_c0_g1_i2,675,525.785,15.52500,1.574440
2,TRINITY_DN5046_c0_g1_i1,595,445.808,20.00000,2.392140
3,TRINITY_DN5074_c0_g1_i1,583,433.808,18.00000,2.212480
4,TRINITY_DN5003_c0_g1_i1,685,535.785,4.16427,0.414431


In [ ]:
#juntar as tabelas 'trinotate' e 'abundancia' pela coluna 'trascript_id'
trinotate_edit = pd.merge(trinotate, abundancia, on="transcript_id")

#agora a tabela nova 'trinotate_edit' tem os dados das duas tabelas anteriores
trinotate_edit.head()

,#gene_id,transcript_id,sprot_Top_BLASTX_hit,RNAMMER,prot_id,prot_coords,sprot_Top_BLASTP_hit,Pfam,SignalP,TmHMM,eggnog,Kegg,gene_ontology_blast,gene_ontology_pfam,transcript,peptide,length,eff_length,est_counts,tpm
0,TRINITY_DN5078_c0_g1,TRINITY_DN5078_c0_g1_i1,.,.,TRINITY_DN5078_c0_g1_i1.p1,2-628[+],.,.,.,.,.,.,.,.,.,.,629,479.808,11.47500,1.275230
1,TRINITY_DN5078_c0_g1,TRINITY_DN5078_c0_g1_i2,.,.,TRINITY_DN5078_c0_g1_i2.p1,2-673[+],.,.,.,.,.,.,.,.,.,.,675,525.785,15.52500,1.574440
2,TRINITY_DN5046_c0_g1,TRINITY_DN5046_c0_g1_i1,"PAPOB_HUMAN^PAPOB_HUMAN^Q:263-15,H:61-146^38.3...",.,.,.,.,.,.,.,COG5186^polyA polymerase,KEGG:hsa:56903`KO:K14376,GO:0005737^cellular_component^cytoplasm`GO:000...,.,.,.,595,445.808,20.00000,2.392140
3,TRINITY_DN5074_c0_g1,TRINITY_DN5074_c0_g1_i1,.,.,TRINITY_DN5074_c0_g1_i1.p1,6-335[-],.,.,.,.,.,.,.,.,.,.,583,433.808,18.00000,2.212480
4,TRINITY_DN5003_c0_g1,TRINITY_DN5003_c0_g1_i1,.,.,TRINITY_DN5003_c0_g1_i1.p1,71-604[+],.,.,.,.,.,.,.,.,.,.,685,535.785,4.16427,0.414431


In [ ]:
#fazer uma serie com os codigos uniprot
#a partir desses codigos sera possivel identificar a classe enzimatica
codigos_uniprot = trinotate["sprot_Top_BLASTP_hit"]
codigos_uniprot

0                                                        .
1                                                        .
2                                                        .
3                                                        .
4                                                        .
                               ...                        
69075    GEFP_DICDI^GEFP_DICDI^Q:443-673,H:1269-1498^37...
69076    GEFA_DICDI^GEFA_DICDI^Q:436-592,H:348-509^35.5...
69077    GEFP_DICDI^GEFP_DICDI^Q:674-904,H:1269-1498^37...
69078    GEFP_DICDI^GEFP_DICDI^Q:298-528,H:1269-1498^37...
69079    ATC4_YEAST^ATC4_YEAST^Q:18-1356,H:176-1459^43....
Name: sprot_Top_BLASTP_hit, Length: 69080, dtype: object

In [ ]:
#aqui o texto de cada linha da serie sera dividido em varias colunas
#em todos os caracteres '^'
lista_uniprot = pd.DataFrame(r.split("^") for r in codigos_uniprot)
lista_uniprot

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84
0,.,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,.,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,.,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,.,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,.,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69075,GEFP_DICDI,GEFP_DICDI,"Q:443-673,H:1269-1498",37.07%ID,E:2e-36,RecName: Full=Ras guanine nucleotide exchange ...,Eukaryota; Amoebozoa; Mycetozoa; Dictyostelids...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
69076,GEFA_DICDI,GEFA_DICDI,"Q:436-592,H:348-509",35.58%ID,E:2e-24,RecName: Full=Ras guanine nucleotide exchange ...,Eukaryota; Amoebozoa; Mycetozoa; Dictyostelids...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
69077,GEFP_DICDI,GEFP_DICDI,"Q:674-904,H:1269-1498",37.07%ID,E:3e-36,RecName: Full=Ras guanine nucleotide exchange ...,Eukaryota; Amoebozoa; Mycetozoa; Dictyostelids...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,

In [ ]:
#selecionar a primeira coluna da tabela criada com os codigos uniprot
#cada codigo entra apenas uma vez nessa serie
uniprot = np.unique(lista_uniprot[0])

#retirar os '.' da serie para sobrar apenas os codigos
uniprot_novo = np.delete(uniprot, np.where(uniprot == "."))

In [ ]:
#salvar um arquivo de texto novo com os codigos uniprot
#o aquivo 'codigos_uniprot.txt' sera baixado no computador
f = open("codigos_uniprot.txt", "w")
for i in uniprot_novo:
  f.write(f'{i}\n')
f.close()
files.download("codigos_uniprot.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

O arquivo criado anteriormente deve ser usado na ferramenta **Retrieve/ID mapping** do Uniprot , no site:<br>https://www.uniprot.org/uploadlists/

1. Clique em ```Browse...``` e selecione o aquivo ```codigos_uniprot.txt```
2. As opções devem ser ```From - UniprotKB AC/ID``` e ```To - UniprotKB```
3. Clique em ```Submit```
4. Quando finalizar o processo, clique em ```Columns```. Selecione todas as colunas de interesse e clique em ```Save```<br>
**IMPORTANTE: a coluna ```EC number``` deve ser selecionada**<br>
5. Clique em ```Download```, selecione o formato ```Tab-separated``` e ```Go```
6. Faça upload do arquivo baixado no google drive


In [ ]:
#criando uma tabela nova com apenas uma coluna dos codigos uniprot
entry_name = pd.DataFrame(lista_uniprot[0])
entry_name.columns = ["Entry name"]

#conferindo o tamanho da tabela
entry_name.shape

(69080, 1)

In [ ]:
#adicionando a tabela criada anteriormente na tabela principal
trinotate_edit1 = trinotate_edit.join(entry_name)
trinotate_edit1.head()

,#gene_id,transcript_id,sprot_Top_BLASTX_hit,RNAMMER,prot_id,prot_coords,sprot_Top_BLASTP_hit,Pfam,SignalP,TmHMM,eggnog,Kegg,gene_ontology_blast,gene_ontology_pfam,transcript,peptide,length,eff_length,est_counts,tpm,Entry name
0,TRINITY_DN5078_c0_g1,TRINITY_DN5078_c0_g1_i1,.,.,TRINITY_DN5078_c0_g1_i1.p1,2-628[+],.,.,.,.,.,.,.,.,.,.,629,479.808,11.47500,1.275230,.
1,TRINITY_DN5078_c0_g1,TRINITY_DN5078_c0_g1_i2,.,.,TRINITY_DN5078_c0_g1_i2.p1,2-673[+],.,.,.,.,.,.,.,.,.,.,675,525.785,15.52500,1.574440,.
2,TRINITY_DN5046_c0_g1,TRINITY_DN5046_c0_g1_i1,"PAPOB_HUMAN^PAPOB_HUMAN^Q:263-15,H:61-146^38.3...",.,.,.,.,.,.,.,COG5186^polyA polymerase,KEGG:hsa:56903`KO:K14376,GO:0005737^cellular_component^cytoplasm`GO:000...,.,.,.,595,445.808,20.00000,2.392140,.
3,TRINITY_DN5074_c0_g1,TRINITY_DN5074_c0_g1_i1,.,.,TRINITY_DN5074_c0_g1_i1.p1,6-335[-],.,.,.,.,.,.,.,.,.,.,583,433.808,18.00000,2.212480,.
4,TRINITY_DN5003_c0_g1,TRINITY_DN5003_c0_g1_i1,.,.,TRINITY_DN5003_c0_g1_i1.p1,71-604[+],.,.,.,.,.,.,.,.,.,.,685,535.785,4.16427,0.414431,.


In [ ]:
#carregando a tabela que foi baixada do uniprot
#o nome do aquivo baixado do uniprot aqui se chama 'RMA_uniprot.tab'
#substitua pelo endereço do arquivo com o nome que salvou
rma_uniprot = pd.read_csv("/content/drive/MyDrive/RMA/RMA_uniprot.tab", sep="\t")
rma_uniprot
#repare que nao tem o mesmo numero de linhas que a tabela principal

,Entry,Entry name,EC number,Active site,Pathway,pH dependence,Temperature dependence,Allergenic properties,Biotechnological use,Disruption phenotype,Involvement in disease,Mutagenesis,Pharmaceutical use,Toxic dose
0,P93212,14337_SOLLC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,P92177,1433E_DROME,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MUTAGEN 183; /note=""E->K: Suppressor of sev-R...",NaN,NaN
2,Q99002,1433_TRIHA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Q8GYY0,1A112_ARATH,2.6.1.-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Q6PD03,2A5A_MOUSE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11362,O48626,ZW10_ARATH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DISRUPTION PHENOTYPE: Accumulation of the prec...,NaN,NaN,NaN,NaN
11363,O43264,ZW10_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11364,Q5SSH7,ZZEF1_MOUSE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11365,Q8IYH5,ZZZ3_HUMAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#mesclar as tabelas do uniprot com a tabela principal pela coluna 'Entry name'
trinotate_edit2 = pd.merge(trinotate_edit1, rma_uniprot, on="Entry name")
trinotate_edit2
#o numero de linhas eh menor pois nem todas as linhas iniciais tem uma proteina
#correspondente no banco de dados do uniprot
#aquelas sequencias que tinham um '.' no lugar do codigo uniprot
#nao aparecem nessa nova tabela

,#gene_id,transcript_id,sprot_Top_BLASTX_hit,RNAMMER,prot_id,prot_coords,sprot_Top_BLASTP_hit,Pfam,SignalP,TmHMM,eggnog,Kegg,gene_ontology_blast,gene_ontology_pfam,transcript,peptide,length,eff_length,est_counts,tpm,Entry name,Entry,EC number,Active site,Pathway,pH dependence,Temperature dependence,Allergenic properties,Biotechnological use,Disruption phenotype,Involvement in disease,Mutagenesis,Pharmaceutical use,Toxic dose
0,TRINITY_DN5043_c0_g1,TRINITY_DN5043_c0_g1_i1,"YDQC_SCHPO^YDQC_SCHPO^Q:137-340,H:216-283^44.1...",.,TRINITY_DN5043_c0_g1_i1.p1,2-472[+],"YDQC_SCHPO^YDQC_SCHPO^Q:46-119,H:216-292^46.75...",.,.,.,.,KEGG:spo:SPAC5D6.12,GO:0005739^cellular_component^mitochondrion`GO...,.,.,.,556,406.827,76.0000,9.961120,YDQC_SCHPO,O14204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRINITY_DN3416_c0_g1,TRINITY_DN3416_c0_g1_i1,"YDQC_SCHPO^YDQC_SCHPO^Q:575-937,H:169-290^30.6...",.,TRINITY_DN3416_c0_g1_i1.p1,2-1060[+],"YDQC_SCHPO^YDQC_SCHPO^Q:192-312,H:169-290^30.6...",.,.,.,.,KEGG:spo:SPAC5D6.12,GO:0005739^cellular_component^mitochondrion`GO...,.,.,.,1148,998.785,82.0000,4.377700,YDQC_SCHPO,O14204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TRINITY_DN5061_c0_g1,TRINITY_DN5061_c0_g1_i1,"ACO13_MOUSE^ACO13_MOUSE^Q:127-483,H:22-137^30%...",.,TRINITY_DN5061_c0_g1_i1.p1,1-513[+],"ACO13_MOUSE^ACO13_MOUSE^Q:43-161,H:22-137^30%I...",.,.,.,COG2050^thioesterase Superfamily protein,KEGG:mmu:66834`KO:K17362,GO:0005829^cellular_component^cytosol`GO:00057...,.,.,.,672,522.785,97.0000,9.893560,ACO13_MOUSE,Q9CQR4,3.1.2.-; 3.1.2.2,NaN,NaN,NaN,NaN,NaN,NaN,DISRUPTION PHENOTYPE: No visible phenotype unt...,NaN,NaN,NaN,NaN
3,TRINITY_DN5028_c0_g1,TRINITY_DN5028_c0_g1_i1,"RPA12_YEAST^RPA12_YEAST^Q:44-406,H:4-125^50.81...",.,TRINITY_DN5028_c0_g1_i1.p1,2-409[+],"RPA12_YEAST^RPA12_YEAST^Q:15-135,H:4-125^50.81...",.,sigP:1^12^0.505^YES,.,.,KEGG:sce:YJR063W`KO:K03000,GO:0005736^cellular_component^RNA polymerase I...,.,.,.,503,353.863,96.7926,14.585100,RPA12_YEAST,P32529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MUTAGEN 10; /note=""C->S: Severe growth defect...",NaN,NaN
4,TRINITY_DN5028_c0_g1,TRINITY_DN5028_c0_g1_i2,"RPA12_SCHPO^RPA12_SCHPO^Q:339-467,H:76-118^72....",.,TRINITY_DN5028_c0_g1_i2.p1,2-325[+],"RPA12_YEAST^RPA12_YEAST^Q:15-97,H:4-87^37.21%I...",.,sigP:1^12^0.505^YES,.,.,KEGG:sce:YJR063W`KO:K03000,GO:0005736^cellular_component^RNA polymerase I...,.,.,.,567,417.808,18.2074,2.323680,RPA12_YEAST,P32529,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MUTAGEN 10; /note=""C->S: Severe growth defect...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44678,TRINITY_DN21052_c0_g1,TRINITY_DN21052_c0_g1_i5,"DMC1_HUMAN^DMC1_HUMAN^Q:175-801,H:129-337^66.0...",.,TRINITY_DN21052_c0_g1_i5.p1,388-813[+],"DMC1_MOUSE^DMC1_MOUSE^Q:1-139,H:200-338^70.5%I...",.,.,.,COG0468^Can catalyze the hydrolysis of ATP in ...,KEGG:mmu:13404`KO:K10872,GO:0005694^cellular_component^chromosome`GO:00...,.,.,.,880,730.785,86.4768,6.309770,DMC1_MOUSE,Q61880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44679,TRINITY_DN21000_c0_g1,TRINITY_DN21000_c0_g1_i1,"GEFP_DICDI^GEFP_DICDI^Q:2287-3105,H:1265-1498^...",.,TRINITY_DN21000_c0_g1_i1.p1,1408-2772[+],"GEFA_DICDI^GEFA_DICDI^Q:291-447,H:348-509^35.5...",.,.,.,ENOG410XR96^Son of sevenless homolog,KEGG:ddi:DDB_G0284329,GO:0005829^cellular_component^cytosol`GO:00058...,.,.,.,3285,3135.790,38.0536,0.647072,GEFA_DICDI,Q54PQ4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"DISRUPTION PHENOTYPE: Grows at a normal rate, ...",NaN,NaN,NaN,NaN
44680,TRINITY_DN21000_c0_g1,TRINITY_DN21000_c0_g1_i3,"GEFP_DICDI^GEFP_DICDI^Q:2141-2959,H:1265-1498^...",.,TRINITY_DN21000_c0_g1_i3.p1,134-2626[+],"GEFA_DICDI^GEFA_DICDI^Q:667-823,H:348-509^35.5...",.,.,.,ENOG410XR96^Son of sevenless homolog,KEGG:ddi:DDB_G0284329,GO:0005829^cellular_component^cytosol`GO:00058...,.,.,.,3139,2989.790,94.8339,1.691330,GEFA_DICDI,Q54PQ4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"DISRUPTION PHENOTYPE: Grows at a normal ra

In [ ]:
#daqui pra frente sera selecionado apenas as sequencias de interesse
#comecando por selecionar todas as hidrolases
hydrolases = pd.DataFrame(trinotate_edit2[trinotate_edit2["EC number"].str.match("^3.*") == True])
hydrolases.head()

#se quiser selecionar outra classe basta trocar o numero '3' pelo numero da classe desejada
#recomendo trocar o nome da tabela de 'hydrolases' pelo nome correspondente

,#gene_id,transcript_id,sprot_Top_BLASTX_hit,RNAMMER,prot_id,prot_coords,sprot_Top_BLASTP_hit,Pfam,SignalP,TmHMM,eggnog,Kegg,gene_ontology_blast,gene_ontology_pfam,transcript,peptide,length,eff_length,est_counts,tpm,Entry name,Entry,EC number,Active site,Pathway,pH dependence,Temperature dependence,Allergenic properties,Biotechnological use,Disruption phenotype,Involvement in disease,Mutagenesis,Pharmaceutical use,Toxic dose
2,TRINITY_DN5061_c0_g1,TRINITY_DN5061_c0_g1_i1,"ACO13_MOUSE^ACO13_MOUSE^Q:127-483,H:22-137^30%...",.,TRINITY_DN5061_c0_g1_i1.p1,1-513[+],"ACO13_MOUSE^ACO13_MOUSE^Q:43-161,H:22-137^30%I...",.,.,.,COG2050^thioesterase Superfamily protein,KEGG:mmu:66834`KO:K17362,GO:0005829^cellular_component^cytosol`GO:00057...,.,.,.,672,522.785,97.00,9.89356,ACO13_MOUSE,Q9CQR4,3.1.2.-; 3.1.2.2,NaN,NaN,NaN,NaN,NaN,NaN,DISRUPTION PHENOTYPE: No visible phenotype unt...,NaN,NaN,NaN,NaN
126,TRINITY_DN5045_c0_g1,TRINITY_DN5045_c0_g1_i1,"CDA_AMYRO^CDA_AMYRO^Q:1-564,H:175-359^65.43%ID...",.,TRINITY_DN5045_c0_g1_i1.p1,1-744[+],"CDA_AMYRO^CDA_AMYRO^Q:1-188,H:175-359^67.02%ID...",.,.,ExpAA=19.04^PredHel=1^Topology=i224-246o,.,.,GO:0004099^molecular_function^chitin deacetyla...,.,.,.,841,691.785,7552.48,582.13300,CDA_AMYRO,P50325,3.5.1.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
127,TRINITY_DN1513_c0_g1,TRINITY_DN1513_c0_g1_i1,"CDA_AMYRO^CDA_AMYRO^Q:5-472,H:211-358^42.24%ID...",.,TRINITY_DN1513_c0_g1_i1.p1,2-478[+],"CDA_AMYRO^CDA_AMYRO^Q:2-157,H:211-358^42.24%ID...",.,.,.,.,.,GO:0004099^molecular_function^chitin deacetyla...,.,.,.,505,355.863,14.00,2.09772,CDA_AMYRO,P50325,3.5.1.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,TRINITY_DN1589_c0_g1,TRINITY_DN1589_c0_g1_i1,"CDA_AMYRO^CDA_AMYRO^Q:218-868,H:67-281^47.51%I...",.,TRINITY_DN1589_c0_g1_i1.p1,2-868[+],"CDA_AMYRO^CDA_AMYRO^Q:73-289,H:67-281^47.51%ID...",.,sigP:1^27^0.69^YES,.,.,.,GO:0004099^molecular_function^chitin deacetyla...,.,.,.,868,718.785,45.00,3.33824,CDA_AMYRO,P50325,3.5.1.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,TRINITY_DN21788_c0_g1,TRINITY_DN21788_c0_g1_i1,"CDA_AMYRO^CDA_AMYRO^Q:6-698,H:145-358^42.49%ID...",.,TRINITY_DN21788_c0_g1_i1.p1,3-812[+],"CDA_AMYRO^CDA_AMYRO^Q:2-232,H:145-358^42.49%ID...",.,.,.,.,.,GO:0004099^molecular_function^chitin deacetyla...,.,.,.,833,683.785,23.00,1.79354,CDA_AMYRO,P50325,3.5.1.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#aqui esta sendo selecionado as peptidases, usando numero EC '3.4'
peptidases = pd.DataFrame(hydrolases[hydrolases["EC number"].str.match("^3.4.*")==True])
peptidases.head()

,#gene_id,transcript_id,sprot_Top_BLASTX_hit,RNAMMER,prot_id,prot_coords,sprot_Top_BLASTP_hit,Pfam,SignalP,TmHMM,eggnog,Kegg,gene_ontology_blast,gene_ontology_pfam,transcript,peptide,length,eff_length,est_counts,tpm,Entry name,Entry,EC number,Active site,Pathway,pH dependence,Temperature dependence,Allergenic properties,Biotechnological use,Disruption phenotype,Involvement in disease,Mutagenesis,Pharmaceutical use,Toxic dose
675,TRINITY_DN12957_c0_g1,TRINITY_DN12957_c0_g1_i1,"TASP1_HUMAN^TASP1_HUMAN^Q:29-850,H:43-330^45.1...",.,TRINITY_DN12957_c0_g1_i1.p1,2-853[+],"TASP1_HUMAN^TASP1_HUMAN^Q:10-283,H:43-330^45.1...",.,.,.,COG1446^asparaginase,KEGG:hsa:55617`KO:K08657,GO:0005737^cellular_component^cytoplasm`GO:001...,.,.,.,853,703.785,48.57330,3.680120,TASP1_HUMAN,Q9H6P5,3.4.25.-,"ACT_SITE 234; /note=""Nucleophile""; /evidence...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MUTAGEN 233; /note=""D->A: 0.1% enzymatic acti...",NaN,NaN
676,TRINITY_DN12957_c0_g1,TRINITY_DN12957_c0_g1_i4,"TASP1_HUMAN^TASP1_HUMAN^Q:29-694,H:43-276^43.8...",.,TRINITY_DN12957_c0_g1_i4.p1,2-703[+],"TASP1_HUMAN^TASP1_HUMAN^Q:10-231,H:43-276^43.8...",.,.,.,COG1446^asparaginase,KEGG:hsa:55617`KO:K08657,GO:0005737^cellular_component^cytoplasm`GO:001...,.,.,.,724,574.785,8.35351,0.774938,TASP1_HUMAN,Q9H6P5,3.4.25.-,"ACT_SITE 234; /note=""Nucleophile""; /evidence...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MUTAGEN 233; /note=""D->A: 0.1% enzymatic acti...",NaN,NaN
677,TRINITY_DN5718_c0_g1,TRINITY_DN5718_c0_g1_i3,"TASP1_HUMAN^TASP1_HUMAN^Q:226-579,H:185-314^44...",.,TRINITY_DN5718_c0_g1_i3.p1,217-585[+],"TASP1_HUMAN^TASP1_HUMAN^Q:4-121,H:185-314^44.7...",.,sigP:1^17^0.608^YES,.,COG1446^asparaginase,KEGG:hsa:55617`KO:K08657,GO:0005737^cellular_component^cytoplasm`GO:001...,.,.,.,586,436.808,8.17668,0.998137,TASP1_HUMAN,Q9H6P5,3.4.25.-,"ACT_SITE 234; /note=""Nucleophile""; /evidence...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MUTAGEN 233; /note=""D->A: 0.1% enzymatic acti...",NaN,NaN
678,TRINITY_DN7132_c0_g1,TRINITY_DN7132_c0_g1_i1,"TASP1_HUMAN^TASP1_HUMAN^Q:53-1000,H:43-395^43....",.,TRINITY_DN7132_c0_g1_i1.p1,41-1045[+],"TASP1_HUMAN^TASP1_HUMAN^Q:5-320,H:43-395^43.66...",.,.,.,COG1446^asparaginase,KEGG:hsa:55617`KO:K08657,GO:0005737^cellular_component^cytoplasm`GO:001...,.,.,.,1251,1101.790,74.00000,3.581280,TASP1_HUMAN,Q9H6P5,3.4.25.-,"ACT_SITE 234; /note=""Nucleophile""; /evidence...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MUTAGEN 233; /note=""D->A: 0.1% enzymatic acti...",NaN,NaN
679,TRINITY_DN12957_c0_g1,TRINITY_DN12957_c0_g1_i2,"TASP1_MOUSE^TASP1_MOUSE^Q:238-846,H:113-330^45...",.,TRINITY_DN12957_c0_g1_i2.p1,259-849[+],"TASP1_MOUSE^TASP1_MOUSE^Q:1-196,H:120-330^43.8...",.,.,.,COG1446^asparaginase,KEGG:mmu:75812`KO:K08657,GO:0005737^cellular_component^cytoplasm`GO:001...,.,.,.,849,699.785,52.77060,4.020980,TASP1_MOUSE,Q8R1G1,3.4.25.-,"ACT_SITE 234; /note=""Nucleophile""; /evidence...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#selecao das omega peptidases, numero EC '3.4.19'
omega_pep = pd.DataFrame(hydrolases[hydrolases["EC number"].str.match("^3.4.19.*")==True])
omega_pep.reset_index(inplace=True)
omega_pep

,index,#gene_id,transcript_id,sprot_Top_BLASTX_hit,RNAMMER,prot_id,prot_coords,sprot_Top_BLASTP_hit,Pfam,SignalP,TmHMM,eggnog,Kegg,gene_ontology_blast,gene_ontology_pfam,transcript,peptide,length,eff_length,est_counts,tpm,Entry name,Entry,EC number,Active site,Pathway,pH dependence,Temperature dependence,Allergenic properties,Biotechnological use,Disruption phenotype,Involvement in disease,Mutagenesis,Pharmaceutical use,Toxic dose
0,2559,TRINITY_DN10581_c0_g1,TRINITY_DN10581_c0_g1_i1,"UBP2_SCHPO^UBP2_SCHPO^Q:990-2993,H:473-1100^29...",.,TRINITY_DN10581_c0_g1_i1.p1,99-3014[+],"UBP2_SCHPO^UBP2_SCHPO^Q:298-965,H:473-1100^28....",.,.,.,.,KEGG:spo:SPAC328.06`KO:K11849,GO:0005737^cellular_component^cytoplasm`GO:000...,.,.,.,3261,3111.790,135.7220,2.325640,UBP2_SCHPO,Q9P3U0,3.4.19.12,"ACT_SITE 623; /note=""Nucleophile""; /evidence...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2560,TRINITY_DN10581_c0_g1,TRINITY_DN10581_c0_g1_i3,"UBP2_SCHPO^UBP2_SCHPO^Q:493-2565,H:473-1123^30...",.,TRINITY_DN10581_c0_g1_i3.p1,1-2628[+],"UBP2_SCHPO^UBP2_SCHPO^Q:165-855,H:473-1123^30....",.,.,.,.,KEGG:spo:SPAC328.06`KO:K11849,GO:0005737^cellular_component^cytoplasm`GO:000...,.,.,.,2703,2553.790,217.0030,4.530910,UBP2_SCHPO,Q9P3U0,3.4.19.12,"ACT_SITE 623; /note=""Nucleophile""; /evidence...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2561,TRINITY_DN10581_c0_g1,TRINITY_DN10581_c0_g1_i4,"UBP2_SCHPO^UBP2_SCHPO^Q:990-3062,H:473-1123^30...",.,TRINITY_DN10581_c0_g1_i4.p1,99-3125[+],"UBP2_SCHPO^UBP2_SCHPO^Q:298-990,H:473-1125^30....",.,.,.,.,KEGG:spo:SPAC328.06`KO:K11849,GO:0005737^cellular_component^cytoplasm`GO:000...,.,.,.,3200,3050.790,139.6140,2.440170,UBP2_SCHPO,Q9P3U0,3.4.19.12,"ACT_SITE 623; /note=""Nucleophile""; /evidence...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2562,TRINITY_DN10581_c0_g1,TRINITY_DN10581_c0_g1_i7,"UBP2_SCHPO^UBP2_SCHPO^Q:493-2496,H:473-1100^28...",.,TRINITY_DN10581_c0_g1_i7.p1,1-2517[+],"UBP2_SCHPO^UBP2_SCHPO^Q:165-832,H:473-1100^29....",.,.,.,.,KEGG:spo:SPAC328.06`KO:K11849,GO:0005737^cellular_component^cytoplasm`GO:000...,.,.,.,2764,2614.790,207.8350,4.238250,UBP2_SCHPO,Q9P3U0,3.4.19.12,"ACT_SITE 623; /note=""Nucleophile""; /evidence...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2563,TRINITY_DN11532_c0_g1,TRINITY_DN11532_c0_g1_i1,"UBP2_YEAST^UBP2_YEAST^Q:1634-2215,H:1075-1256^...",.,TRINITY_DN11532_c0_g1_i1.p1,695-2575[+],"UBP2_SCHPO^UBP2_SCHPO^Q:259-458,H:895-1081^26....",.,.,.,.,KEGG:spo:SPAC328.06`KO:K11849,GO:0005737^cellular_component^cytoplasm`GO:000...,.,.,.,2652,2502.790,13.9666,0.297558,UBP2_SCHPO,Q9P3U0,3.4.19.12,"ACT_SITE 623; /note=""Nucleophile""; /evidence...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,43205,TRINITY_DN16732_c0_g1,TRINITY_DN16732_c0_g1_i3,"UBP30_XENTR^UBP30_XENTR^Q:423-1721,H:83-504^26...",.,TRINITY_DN16732_c0_g1_i3.p1,3-1856[+],"UBP30_XENTR^UBP30_XENTR^Q:141-573,H:83-504^26....",.,.,ExpAA=41.33^PredHel=2^Topology=o15-37i42-61o,ENOG410XQQ0^ubiquitin carboxyl-terminal hydrolase,KEGG:xtr:733894`KO:K11851,GO:0005829^cellular_component^cytosol`GO:00160...,.,.,.,1893,1743.790,75.5363,2.309760,UBP30_XENTR,A4QNN3,3.4.19.12,"ACT_SITE 94; /note=""Nucleophile""; /evidence=...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
301,43523,TRINITY_DN11831_c0_g1,TRINITY_DN11831_c0_g1_i3,"UCHL3_MOUSE^UCHL3_MOUSE^Q:58-372,H:5-109^51.43...",.,TRINITY_DN11831_c0_g1_i3.p1,1-474[+],"UCHL3_RAT^UCHL3_RAT^Q:20-124,H:5-109^51.43%ID^...",.,.,.,ENOG4111HNA^ubiquitin carboxyl-terminal,KEGG:rno:498560`KO:K05609,GO:0005737^cellular_component^cytoplasm`GO:000...,.,.,.,822,672.785,89.1806,7.068020,UCHL3_RAT,Q91Y78,3.4.19.12,"ACT_SITE 95; /note=""Nucleophile""; /evidence=...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
302,43826,TRINITY_DN15913_c0_g1,TRINITY_DN15913_c0_g1_i1,"UCH1_ARATH^UCH1_ARATH^Q:100-513,H:3-137^42.75%...",.,TRINITY_DN15913_c0_g1_i1.p1,67-582[+],"UCH1_ARATH^UCH1_ARATH^Q:12-162,H:3-150

In [ ]:
#reorganizando os indices da tabela
omega_pep = omega_pep.drop("index", axis=1)

In [ ]:
#selecao das treonino peptidases, numero EC '3.4.25'
threonine_pep = pd.DataFrame(hydrolases[hydrolases["EC number"].str.match("^3.4.25.*")==True])
threonine_pep.reset_index(inplace=True)
threonine_pep

,index,#gene_id,transcript_id,sprot_Top_BLASTX_hit,RNAMMER,prot_id,prot_coords,sprot_Top_BLASTP_hit,Pfam,SignalP,TmHMM,eggnog,Kegg,gene_ontology_blast,gene_ontology_pfam,transcript,peptide,length,eff_length,est_counts,tpm,Entry name,Entry,EC number,Active site,Pathway,pH dependence,Temperature dependence,Allergenic properties,Biotechnological use,Disruption phenotype,Involvement in disease,Mutagenesis,Pharmaceutical use,Toxic dose
0,675,TRINITY_DN12957_c0_g1,TRINITY_DN12957_c0_g1_i1,"TASP1_HUMAN^TASP1_HUMAN^Q:29-850,H:43-330^45.1...",.,TRINITY_DN12957_c0_g1_i1.p1,2-853[+],"TASP1_HUMAN^TASP1_HUMAN^Q:10-283,H:43-330^45.1...",.,.,.,COG1446^asparaginase,KEGG:hsa:55617`KO:K08657,GO:0005737^cellular_component^cytoplasm`GO:001...,.,.,.,853,703.785,48.57330,3.680120,TASP1_HUMAN,Q9H6P5,3.4.25.-,"ACT_SITE 234; /note=""Nucleophile""; /evidence...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MUTAGEN 233; /note=""D->A: 0.1% enzymatic acti...",NaN,NaN
1,676,TRINITY_DN12957_c0_g1,TRINITY_DN12957_c0_g1_i4,"TASP1_HUMAN^TASP1_HUMAN^Q:29-694,H:43-276^43.8...",.,TRINITY_DN12957_c0_g1_i4.p1,2-703[+],"TASP1_HUMAN^TASP1_HUMAN^Q:10-231,H:43-276^43.8...",.,.,.,COG1446^asparaginase,KEGG:hsa:55617`KO:K08657,GO:0005737^cellular_component^cytoplasm`GO:001...,.,.,.,724,574.785,8.35351,0.774938,TASP1_HUMAN,Q9H6P5,3.4.25.-,"ACT_SITE 234; /note=""Nucleophile""; /evidence...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MUTAGEN 233; /note=""D->A: 0.1% enzymatic acti...",NaN,NaN
2,677,TRINITY_DN5718_c0_g1,TRINITY_DN5718_c0_g1_i3,"TASP1_HUMAN^TASP1_HUMAN^Q:226-579,H:185-314^44...",.,TRINITY_DN5718_c0_g1_i3.p1,217-585[+],"TASP1_HUMAN^TASP1_HUMAN^Q:4-121,H:185-314^44.7...",.,sigP:1^17^0.608^YES,.,COG1446^asparaginase,KEGG:hsa:55617`KO:K08657,GO:0005737^cellular_component^cytoplasm`GO:001...,.,.,.,586,436.808,8.17668,0.998137,TASP1_HUMAN,Q9H6P5,3.4.25.-,"ACT_SITE 234; /note=""Nucleophile""; /evidence...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MUTAGEN 233; /note=""D->A: 0.1% enzymatic acti...",NaN,NaN
3,678,TRINITY_DN7132_c0_g1,TRINITY_DN7132_c0_g1_i1,"TASP1_HUMAN^TASP1_HUMAN^Q:53-1000,H:43-395^43....",.,TRINITY_DN7132_c0_g1_i1.p1,41-1045[+],"TASP1_HUMAN^TASP1_HUMAN^Q:5-320,H:43-395^43.66...",.,.,.,COG1446^asparaginase,KEGG:hsa:55617`KO:K08657,GO:0005737^cellular_component^cytoplasm`GO:001...,.,.,.,1251,1101.790,74.00000,3.581280,TASP1_HUMAN,Q9H6P5,3.4.25.-,"ACT_SITE 234; /note=""Nucleophile""; /evidence...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"MUTAGEN 233; /note=""D->A: 0.1% enzymatic acti...",NaN,NaN
4,679,TRINITY_DN12957_c0_g1,TRINITY_DN12957_c0_g1_i2,"TASP1_MOUSE^TASP1_MOUSE^Q:238-846,H:113-330^45...",.,TRINITY_DN12957_c0_g1_i2.p1,259-849[+],"TASP1_MOUSE^TASP1_MOUSE^Q:1-196,H:120-330^43.8...",.,.,.,COG1446^asparaginase,KEGG:mmu:75812`KO:K08657,GO:0005737^cellular_component^cytoplasm`GO:001...,.,.,.,849,699.785,52.77060,4.020980,TASP1_MOUSE,Q8R1G1,3.4.25.-,"ACT_SITE 234; /note=""Nucleophile""; /evidence...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,680,TRINITY_DN12957_c0_g1,TRINITY_DN12957_c0_g1_i3,"TASP1_HUMAN^TASP1_HUMAN^Q:238-693,H:113-277^43...",.,TRINITY_DN12957_c0_g1_i3.p1,259-699[+],"TASP1_MOUSE^TASP1_MOUSE^Q:1-145,H:120-277^41.2...",.,.,.,COG1446^asparaginase,KEGG:mmu:75812`KO:K08657,GO:0005737^cellular_component^cytoplasm`GO:001...,.,.,.,720,570.785,9.30256,0.869028,TASP1_MOUSE,Q8R1G1,3.4.25.-,"ACT_SITE 234; /note=""Nucleophile""; /evidence...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,681,TRINITY_DN5718_c0_g1,TRINITY_DN5718_c0_g1_i1,"TASP1_MOUSE^TASP1_MOUSE^Q:2-451,H:127-292^45.1...",.,TRINITY_DN5718_c0_g1_i1.p1,2-478[+],"TASP1_MOUSE^TASP1_MOUSE^Q:1-157,H:127-299^43.9...",.,.,.,COG1446^asparaginase,KEGG:mmu:75812`KO:K08657,GO:0005737^cellular_component^cytoplasm`GO:001...,.,.,.,577,427.808,7.91322,0.986299,TASP1_MOUSE,Q8R1G1,3.4.25.-,"ACT_SITE 234; /note=""Nucleophile""; /evidence...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,682,TRINITY_DN5718_c0_g1,TRINITY_DN5718_c0_g1_i2,"TASP1_MOUSE^TASP1_MOUSE^Q:2-517,H:127-314^45.2...",.,TRINITY_DN5718_c0_g1_i2.p1,2-523[+],"TASP1_MOUSE^TASP1_MOUSE^Q:1-172,H:127-

In [ ]:
#reorganizando os indices da tabela
threonine_pep = threonine_pep.drop("index", axis=1)

In [ ]:
#para salvar as tabelas das enzimas selecionadas como ducumento excel '.xls'
#a tabela sera exportada para a lista de arquivos 'Files'do proprio colab

#o nome do arquivo criado para a tabela de omega peptidases esta entre aspas
omega_pep.to_excel(excel_writer="RMA_omegapep.xls", index=False)

#assim como anterionmente sera feito para a tabela de treonino peptidases
threonine_pep.to_excel(excel_writer="RMA_threoninepep.xls", index=False)

#para salvar as tabelas clique nos tres pontinhos a direita do nome e clique em 'Download'